In [1]:
// imports & setups
import {Real} from 'quantlib/types';
import {UnaryFunction} from 'quantlib/function';
import {ISolver1D} from 'quantlib/math/solver1d';

class F1 implements UnaryFunction<Real, Real> {
    f(x: Real): Real { return x*x-1.0; }
    d(x: Real): Real { return 2.0*x; }
}

class F2 implements UnaryFunction<Real, Real> {
    f(x: Real): Real { return 1.0-x*x; }
    d(x: Real): Real { return -2.0*x; }
}

class F3 implements UnaryFunction<Real, Real> {
    f(x: Real): Real { return Math.atan(x-1); }
    d(x: Real): Real { return 1.0 / (1.0+(x-1.0)*(x-1.0)); }
}

function test_not_bracketed(solver: ISolver1D, 
                            name: string, 
                            f: UnaryFunction<Real, Real>, 
                            guess: Real): void {
    let accuracy: Array<number> = [ 1.0e-4, 1.0e-6, 1.0e-8 ];
    let expected: Real = 1.0;
    for(let i=0;i<accuracy.length;i++){
      let root: Real = solver.solve1(f, accuracy[i], guess, 0.1);
      if (Math.abs(root-expected) > accuracy[i]){
        throw new Error(`solver (not bracketed):
                                       expected: ${expected}
                                     calculated: ${root}
                                       accuracy: ${accuracy[i]}`);
      }
    }
}

function test_bracketed(solver: ISolver1D, 
                        name: string, 
                        f: UnaryFunction<Real, Real>, 
                        guess: Real): void {
    let accuracy: Array<number> = [ 1.0e-4, 1.0e-6, 1.0e-8 ];
    let expected: Real = 1.0;
    for(let i=0;i<accuracy.length;i++){
      // guess on the left side of the root, increasing function
      let root: Real = solver.solve2(f, accuracy[i], guess, 0.0, 2.0);
      if (Math.abs(root-expected) > accuracy[i]){
        throw new Error(`solver (bracketed):
                                   expected: ${expected}
                                 calculated: ${root}
                                   accuracy: ${accuracy[i]}`);
      }
    }
}

interface byRef {
    result: Real;
}

class Probe implements UnaryFunction<Real, Real> {

    constructor(result: byRef, offset: Real){
      this._result = result;
      this._offset = offset;
      this._previous = result.result;
    }

    f(x: Real): Real {
      this._result.result = x;
      return this._previous + this._offset - x*x;
    }

    d(x: Real): Real {
      return 2.0*x;
    }

    dummy(): Real {return this._result.result;}

    private _result: byRef;
    private _previous: Real;
    private _offset: Real;
}

function test_last_call_with_root(solver: ISolver1D, 
                                  name: string, 
                                  bracketed: boolean, 
                                  accuracy: Real): void {
    let mins: Array<Real> = [ 3.0, 2.25, 1.5, 1.0 ];
    let maxs: Array<Real> = [ 7.0, 5.75, 4.5, 3.0 ];
    let steps: Array<Real> = [ 0.2, 0.2, 0.1, 0.1 ];
    let offsets: Array<Real> = [ 25.0, 11.0, 5.0, 1.0 ];
    let guesses: Array<Real> = [ 4.5, 4.5, 2.5, 2.5 ];
    //let expected: Array<Real> = [ 5.0, 4.0, 3.0, 2.0 ];

    let argument: byRef = {result: 0.0};
    let result: Real;

    for(let i=0; i<4; ++i) {
      if(bracketed) {
        result = solver.solve2(new Probe(argument, offsets[i]), accuracy, guesses[i], mins[i], maxs[i]);
      } else {
        result = solver.solve1(new Probe(argument, offsets[i]), accuracy, guesses[i], steps[i]);
      }

      let error: Real = Math.abs(result - argument.result);
      // no floating-point comparison: the solver should have
      // called the function with the very same value it's
      // returning
      if(result != argument.result)
        throw new Error(`${name} solver ${bracketed? 'bracket': 'not bracketed'}
                                   index:   ${i}
                                expected:   ${result}
                              calculated: ${argument.result}
                                   error: ${error}`);
    }
}

function test_solver(solver: ISolver1D, 
                     name: string, 
                     accuracy: Real | any): void {
    // guess on the left side of the root, increasing function
    test_not_bracketed(solver, name, new F1(), 0.5);
    test_bracketed(solver, name, new F1(), 0.5);
    // guess on the right side of the root, increasing function
    test_not_bracketed(solver, name, new F1(), 1.5);
    test_bracketed(solver, name, new F1(), 1.5);
    // guess on the left side of the root, decreasing function
    test_not_bracketed(solver, name, new F2(), 0.5);
    test_bracketed(solver, name, new F2(), 0.5);
    // guess on the right side of the root, decreasing function
    test_not_bracketed(solver, name, new F2(), 1.5);
    test_bracketed(solver, name, new F2(), 1.5);
    // situation where bisection is used in the finite difference
    // newton solver as the first step and where the initial
    // guess is equal to the next estimate (which causes an infinite
    // derivative if we do not handle this case with special care)
    test_not_bracketed(solver, name, new F3(), 1.00001);
    // check that the last function call is made with the root value
    if(accuracy != null) {
        test_last_call_with_root(solver, name, false, accuracy);
        test_last_call_with_root(solver, name, true, accuracy);
    }
}

true

In [2]:
// Testing Brent solver...
import {Brent} from 'quantlib/math/solvers1d/brent';
test_solver(new Brent(), 'Brent', 1.0e-6);

undefined

In [3]:
// Testing bisection solver...
import {Bisection} from 'quantlib/math/solvers1d/bisection';
test_solver(new Bisection(), 'Bisection', 1.0e-6);

undefined

In [4]:
// Testing false-position solver...
import {FalsePosition} from 'quantlib/math/solvers1d/falseposition';
test_solver(new FalsePosition(), 'FalsePosition', 1.0e-6);

undefined

In [5]:
// Testing Newton solver...
import {Newton} from 'quantlib/math/solvers1d/newton';
test_solver(new Newton(), 'Newton', 1.0e-12);

undefined

In [6]:
// Testing Newton-safe solver...
import {NewtonSafe} from 'quantlib/math/solvers1d/newtonsafe';
test_solver(new NewtonSafe(), 'NewtonSafe', 1.0e-9);

undefined

In [7]:
// Testing finite-difference Newton-safe solver...
import {FiniteDifferenceNewtonSafe} from 'quantlib/math/solvers1d/finitedifferencenewtonsafe';
import {QL_EPSILON} from 'quantlib/defines';
test_solver(new FiniteDifferenceNewtonSafe(), 'FiniteDifferenceNewtonSafe', QL_EPSILON);

undefined

In [8]:
// Testing Ridder solver...
import {Ridder} from 'quantlib/math/solvers1d/ridder';
test_solver(new Ridder(), 'Ridder', 1.0e-6);

undefined

In [9]:
// Testing secant solver...
import {Secant} from 'quantlib/math/solvers1d/secant';
test_solver(new Secant(), 'Secant', 1.0e-6);

undefined